# Assignment 1

Using text http://www.gutenberg.org/files/2600/2600-0.txt
1. Make text lowercase and remove all punctuation except spaces and dots.
2. Tokenize text by BPE with vocab_size = 100
3. Train 3-gram language model with laplace smoothing $\delta=1$
4. Using beam search with k=10 generate sequences of length=10 conditioned on provided inputs. Treat dots as terminal tokens.
5. Calculate perplexity of the language model for the first sentence.

In [106]:
text = open('peace.txt', 'r', encoding='utf-8').read()[2:]
len(text)

3227579

In [107]:
import string
import re

def preprocess_text(text):
    # TODO
    # make lowercase
    # replace all punctuation except dots with spaces
    # collapse multiple spaces into one '   ' -> ' '
    text = text.lower()
    text = re.sub('[^\w\. ]',' ',text)
    text = re.sub(' +',' ',text)
    text = re.sub('\.(?! )',' ',text)
    return text

text = preprocess_text(text)

assert len(text) == 3141169

In [108]:
text = text.split('.')
text = [x.strip() for x in text]

In [156]:
from collections import Counter
import nltk
from sklearn.base import TransformerMixin


class BPE(TransformerMixin):
    def __init__(self, vocab_size=100):
        super(BPE, self).__init__()
        self.vocab_size = vocab_size
        # index to token
        self.itos = []
        # token to index
        self.stoi = {}
        
    def fit(self, text):
        """
        fit itos and stoi
        text: list of strings 
        """
        
        # TODO
        # tokenize text by symbols and fill in self.itos and self.stoi
        self.itos = list(set(''.join(text)))
        self.stoi = {j:i for i,j in enumerate(self.itos)}# TODO
        text = [[self.stoi[symb] for symb in string] for string in text]
        
        while len(self.itos) < self.vocab_size:
            # TODO
            # count bigram freqencies in the text
            new_token = self.most_common_bigram(text)# most common bigram in the text
            new_id = len(self.itos)
            self.itos.append(new_token)
            self.stoi[new_token] = new_id
            
            # find occurences of the new_token in the text and replace them with new_id
            text = self.replace_bigram(text, new_token, new_id)
            
        return self
    
    def transform(self, text):
        """
        convert text to a sequence of token ids
        text: list of strings
        """
        text = [[self.stoi[symb] for symb in string] for string in text]# TODO tokenize text by symbols using self.stoi
        for token_id, token in enumerate(self.itos):
            # find occurences of the token in the text and replace them with token_id
            if type(token) == tuple:
                text = self.replace_bigram(text, token, token_id)
        return text
    
    def most_common_bigram(self, text):
        bigrams = [(s[i],s[i+1]) for s in text for i in range(len(s)-1)]
        return Counter(bigrams).most_common(1)[0][0]
    
    def replace_bigram(self, text, bigram, mask):
        for sent_id, sent in enumerate(text):
            i = 0
            while i < len(sent)-1:
                if (sent[i], sent[i+1]) == bigram:
                    text[sent_id][i] = mask
                    text[sent_id].pop(i+1)
                i += 1
        return text
        
    
    def decode_token(self, tok):
        """
        tok: int or tuple
        """
        if tok > len(self.itos):
            return ''
        result = self.itos[tok]
        if type(result) == tuple:
            return self.decode_token(result[0]) + self.decode_token(result[1])
        return result
            
    def decode(self, text):
        """
        convert token ids into text
        """
        return ''.join(map(self.decode_token, text))
        
        
vocab_size = 100
bpe = BPE(vocab_size)
tokenized_text = bpe.fit_transform(text)

In [110]:
assert bpe.decode(tokenized_text[0]) == text[0]

In [128]:
import numpy as np
        
    
start_token = vocab_size
end_token = vocab_size + 1
        
    
class LM:
    def __init__(self, vocab_size, delta=1):
        self.delta = delta
        self.vocab_size = vocab_size + 2
        self.proba = np.full(shape=(self.vocab_size,self.vocab_size,self.vocab_size),
                             fill_value=delta)

    def infer(self, a, b, tau=1):
        """
        return vector of probabilities of size self.vocab for 3-grams which start with (a,b) tokens
        a: first token id
        b: second token id
        tau: temperature
        """
        result = np.array([self.get_proba(a,b,i,tau) for i in range(self.vocab_size)])# TODO
        return result
        
    def get_proba(self, a, b, c, tau=1):
        """
        get probability of 3-gram (a,b,c)
        a: first token id
        b: second token id
        c: third token id
        tau: temperature
        """
        result = self.proba[a][b][c] ** (1/tau)/(self.proba[a][b] ** (1/tau)).sum()
        return result
    
    def fit(self, text):
        """
        train language model on text
        text: list of lists
        """
        # TODO count 3-grams in the text
        for sent in text:
            sent = [start_token] + sent + [end_token]
            for i in range(len(sent)-2):
                self.proba[sent[i]][sent[i+1]][sent[i+2]] += 1
        
        return self
    
lm = LM(vocab_size, 1).fit(tokenized_text)

In [127]:
lm.infer(50,16)

array([2.19683656e-04, 2.19683656e-04, 2.19683656e-04, 1.31810193e-03,
       2.19683656e-03, 2.19683656e-04, 1.75746924e-03, 2.19683656e-04,
       4.39367311e-04, 2.19683656e-04, 2.19683656e-04, 2.19683656e-04,
       2.19683656e-04, 2.19683656e-04, 2.19683656e-04, 2.19683656e-04,
       3.27328647e-02, 2.19683656e-04, 2.19683656e-04, 2.19683656e-04,
       2.19683656e-04, 2.19683656e-04, 2.19683656e-04, 2.70210896e-02,
       2.19683656e-04, 2.19683656e-04, 2.19683656e-04, 8.78734622e-04,
       2.19683656e-04, 6.59050967e-04, 2.19683656e-04, 7.31546573e-02,
       2.19683656e-04, 2.19683656e-04, 2.19683656e-04, 9.77592267e-02,
       2.19683656e-04, 2.19683656e-04, 1.53778559e-02, 2.19683656e-04,
       2.19683656e-04, 2.19683656e-04, 2.19683656e-04, 2.19683656e-04,
       2.19683656e-04, 1.84534271e-02, 2.19683656e-04, 2.19683656e-04,
       4.83304042e-03, 1.97715290e-03, 1.01054482e-02, 2.19683656e-04,
       2.19683656e-04, 2.19683656e-04, 2.19683656e-04, 2.19683656e-04,
      

In [144]:
def beam_search(input_seq, lm, max_len=10, k=5, tau=1):
    """
    generate sequence from language model *lm* conditioned on input_seq
    input_seq: sequence of token ids for conditioning
    lm: language model
    max_len: max generated sequence length
    k: size of beam
    tau: temperature
    """
    
    beam = [((input_seq[-2], input_seq[-1], c),
             lm.get_proba(input_seq[-2], input_seq[-1], c, tau=tau)) for c in top_k(lm.infer(input_seq[-2], input_seq[-1],
                                                                                           tau=tau),k=k)]
    
    for i in range(max_len):
        candidates = []
        candidates_proba = []
        for snt, snt_proba in beam:
            if snt[-1] == end_token:
                candidates.append(snt)
                candidates_proba.append(snt_proba)# TODO process terminal token
            else:    
                proba = lm.infer(snt[-2], snt[-1], tau=tau)# probability vector of the next token
                best_k = top_k(proba, k=k)# top-k most probable tokens
                # TODO update candidates' sequences and corresponding probabilities
                for candidate in best_k:
                    candidates.append(snt+(candidate,))
                    candidates_proba.append(snt_proba*lm.get_proba(snt[-2], snt[-1], candidate, tau=tau))
                
        beam = most_probable(candidates, candidates_proba, k=k)# select top-k most probable sequences from candidates
    return beam

top_k = lambda x, k: np.argsort(x)[::-1][:k]

def most_probable(candidates, probas, k):
    return [(candidates[i], probas[i]) for i in top_k(probas, k=k)]

In [151]:
input1 = 'horse '
input1 = bpe.transform([input1])[0]
result = beam_search(input1, lm, max_len=10, k=10, tau=0.1)
# TODO print decoded generated strings and their probabilities
for seq, proba in result:
    print(bpe.decode(input1)+bpe.decode(seq[2:]), proba)

horse with a smill  0.6841029778083697
horse was not been sa 0.04177810773487709
horse was sold not be 0.020091172233032518
horse when said not  0.01957560537478095
horse the countess mary  0.012419496299872258
horse who had been sa 0.010867744904136322
horse with his fack  0.007780060616618095
horse the counderstand wi 0.007335131282565677
horse the cound him and s 0.007187195139002834
horse the cound him and w 0.005751116965404067


In [152]:
input1 = 'her'
input1 = bpe.transform([input1])[0]
result = beam_search(input1, lm, max_len=10, k=10, tau=0.1)
# TODO print decoded generated strings and their probabilities
for seq, proba in result:
    print(bpe.decode(input1)+bpe.decode(seq[2:]), proba)

here with a smill 0.7304666014228682
here was not been s 0.0487404790163391
here was sold not b 0.022797616216872155
here when said no 0.020902320798071274
here who had been s 0.012678867502004658
here was she was not b 0.009807273664700425
here with his heas 0.009110050541107704
here with his fack 0.008307338307196785
here with his hear  0.007996051526025703
here was not seemp 0.0064405700451872685


In [157]:
input1 = 'what'
input1 = bpe.transform([input1])[0]
result = beam_search(input1, lm, max_len=10, k=10, tau=1)
# TODO print decoded generated strings and their probabilities
for seq, proba in result:
    print(bpe.decode(input1)+bpe.decode(seq[2:]), proba)

what 0.02443609022556391
whated 0.011341031957494033
whated to him 0.0002072659731327774
what theight 5.717776867622187e-05
whated to himself 1.6978380760763765e-05
whated to himself it  1.6938712102523908e-06
whated to himself i  1.4867449897241193e-06
whated to himself sa 9.252974424477093e-07
whated to himself he ha 8.899626578301803e-07
whated to himself he w 8.326777051422607e-07


In [158]:
input1 = 'gun '
input1 = bpe.transform([input1])[0]
result = beam_search(input1, lm, max_len=10, k=10, tau=0.1)
# TODO print decoded generated strings and their probabilities
for seq, proba in result:
    print(bpe.decode(input1)+bpe.decode(seq[2:]), proba)

gun and with a smill 0.24096428447585005
gun been said not  0.1624530112505025
gun and said not been 0.10901816275155284
gun and so mussion of  0.03893029842345072
gun but was not been 0.03675414427294098
gun and so mussion hi 0.03249012096768254
gun and so must been  0.031565015712851884
gun said not been  0.025207649072660532
gun and said not see 0.021695841113877797
gun but was sold no 0.02008458233790684


In [163]:
def perplexity(snt, lm):
    """
    snt: sequence of token ids
    lm: language model
    """
    result = 0
    for i in range(2,len(snt)):
        result += np.log(lm.get_proba(snt[i-2], snt[i-1], snt[i]))
    result = result * -1/(len(snt)-2)
    return 2**result

perplexity(tokenized_text[0], lm)

6.090930305288339